In [1]:
import os
import joblib
import argparse
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import config
import models
#import optuna.integration.lightgbm as lgb          
#import matplotlib.pyplot as plt
#import seaborn as sns

In [2]:

""" Arrival """
# 1st hour prediction with actual previous hour's AAR
# load selected prediction time data
train_data_arrival_raw = pd.read_csv( os.path.join(config.input_dir, 'arrival_train.csv'), index_col = 0)
train_data_arrival = train_data_arrival_raw[train_data_arrival_raw['taf'] == 6].reset_index(drop = True)            # 맨 처음은 6시간 taf 쓰기
train_data_arrival = train_data_arrival.drop('taf', axis = 1)

In [5]:
time = 2
train_data_arrival[150:170]

,label,EAD,EDD,year,month,day,hour,DayName,Arpt_cond,P_Airp,...,WSPD_t,WG_t,VIS_t,WC_t,CLA_1LYR_t,BASE_1LYR_t,CLA_2LYR_t,BASE_2LYR_t,CLA_3LYR_t,BASE_3LYR_t
150,31.0,43.0,15.0,2018,1,7,6,7,1.0,1.0,...,6.0,0.0,7000.0,0,0,400.0,0,400.0,0,400.0
151,32.0,27.0,35.0,2018,1,7,7,7,1.0,1.0,...,6.0,0.0,7000.0,0,0,400.0,0,400.0,0,400.0
152,26.0,27.0,32.0,2018,1,7,8,7,1.0,1.0,...,6.0,0.0,7000.0,0,0,400.0,0,400.0,0,400.0
153,23.0,16.0,44.0,2018,1,7,9,7,1.0,1.0,...,6.0,0.0,7000.0,0,0,400.0,0,400.0,0,400.0
154,20.0,18.0,40.0,2018,1,7,10,7,1.0,1.0,...,6.0,0.0,7000.0,0,2,10.0,4,30.0,7,110.0
155,37.0,35.0,23.0,2018,1,7,11,7,1.0,1.0,...,6.0,0.0,7000.0,0,2,10.0,4,30.0,7,110.0
156,22.0,29.0,36.0,2018,1,7,12,7,1.0,1.0,...,6.0,0.0,7000.0,0,2,10.0,4,30.0,7,110.0
157,25.0,25.0,30.0,2018,1,7,13,7,1.0,1.0,...,6.0,0.0,7000.0,0,2,10.0,4,30.0,7,110.0
158,29.0,27.0,33.0,2018,1,7,14,7,1.0,1.0,...,6.0,0.0,7000.0,0,2,10.0,4,30.0,7,110.0
159,32.0,37.0,24.0,2018,1,7,15,7,1.0,1.0,...,6.0,0.0,7000.0,0,2,10.0,4,30.0,7,110.0
